In [1]:
import pandas as pd
from tqdm import tqdm
import math

In [2]:
#!pip list
#!python -m spacy download es_core_news_sm

In [3]:
data = pd.read_csv('./training_data/tv_ner_textcat_100.csv')
data.head() 

,Title,Marca,start,end,"class(TV, SMART, OTRO)"
0,Smart Tv Noblex Di43x5100x Led Full Hd 43 220v,Noblex,9.0,15.0,1
1,Smart Tv Tcl L42s6500 Led Full Hd 42 220v,Tcl,9.0,12.0,1
2,Tv Samsung Smart Tv Hd 32 Para Repuestos,Samsung,3.0,10.0,0
3,Tv Led Samsung 46 Smart Pantalla Rota,Samsung,7.0,14.0,0
4,Tv 29 Sanyo Vizon,Sanyo,6.0,11.0,0


## Format Data

In [4]:
TRAINING_DATA = []

for index, row in data.iterrows(): 
    text = row['Title']
    start = row['start']
    end = row['end']
    entities = []
    #print(start)
    if math.isnan(start):
        #print('no ent')
        pass
    else:        
        entities.append((int(start), int(end), "MARCA"))            
        
    
    #print(dup_start)
    training_example = (text, {"entities": entities})
    TRAINING_DATA.append(training_example)

print(len(TRAINING_DATA))

102


In [5]:
import spacy
import srsly
from spacy.gold import docs_to_json, biluo_tags_from_offsets, spans_from_biluo_tags
from tqdm import tqdm
#spacy.prefer_gpu()
#TRAIN_DATA = [
#    ("Who is Shaka Khan?", {"entities": [(7, 17, "PERSON")]}),
#    ("I like London and Berlin.", {"entities": [(7, 13, "LOC"), (18, 24, "LOC")]}),
#]

#nlp = spacy.blank("en")
nlp = spacy.load('es_core_news_sm', disable = ['tagger', 'ner'])
#nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp.max_length = 1500000
docs = []
for text, annot in tqdm(TRAINING_DATA):
    if len(text) < 1000000:
        doc = nlp(text)
        tags = biluo_tags_from_offsets(doc, annot['entities'])
        entities = spans_from_biluo_tags(doc, tags)
        doc.ents = entities
        docs.append(doc)

srsly.write_json("./training_data/tv_ner_100.json", [docs_to_json(docs)]) 

100%|██████████| 100/100 [00:00<00:00, 775.07it/s]


# Debug

In [7]:
!python3 -m spacy debug-data es '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/TVs/training_data/tv_ner_100.json' '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/TVs/training_data/tv_ner_100.json' --verbose


=========================== Data format validation ===========================
✔ Corpus is loadable

=============================== Training stats ===============================
Training pipeline: tagger, parser, ner
Starting with blank model 'es'
102 training docs
102 evaluation docs
⚠ 102 training examples also in evaluation data
⚠ Low number of examples to train from a blank model (102)
It's recommended to use at least 2000 examples (minimum 100)

============================== Vocab & Vectors ==============================
ℹ 768 total words in the data (229 unique)
10 most common words: 'Tv' (77), 'Smart' (44), ' ' (30), 'Led' (29), 'Pulgadas'
(23), 'Televisor' (22), 'Hd' (21), 'Samsung' (21), '32' (19), '4k' (16)
ℹ No word vectors present in the model

========================== Named Entity Recognition ==========================
ℹ 1 new label, 0 existing labels
0 missing values (tokens with '-' label)
New: 'MARCA' (95)
✔ Good amount of examples for all labels
✔ Examples withou

# Train

In [8]:
!python -m spacy train es ./models/NUBI-TV-NER-100 '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/TVs/training_data/tv_ner_100.json' '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/TVs/training_data/tv_ner_100.json' --base-model 'es_core_news_sm' -p ner -R
#SAco el modelo de base
#!python -m spacy train en NER-SECOND-EMPTY-VEC-15000 '/home/fede/kaggle_competition_2/spacy_2.3.5/data_for_training/train_sent_12500_format.json' '/home/fede/kaggle_competition_2/spacy_2.3.5/data_for_training/dev_sent_3100_format.json' --vectors '/home/fede/kaggle_competition_2/spacy_2.3.5/Diego/w2v-w11-f7-50-spacy'  -p ner -R

✔ Created output directory: models/NUBI-TV-NER-100
Training pipeline: ['ner']
Starting with base model 'es_core_news_sm'
Replacing component from base model 'ner'
Counting training words (limit=0)
/home/fede/anaconda3/envs/spacy_env/lib/python3.8/site-packages/spacy/language.py:635: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  proc.begin_training(

Itn  NER Loss   NER P   NER R   NER F   Token %  CPU WPS
---  ---------  ------  ------  ------  -------  -------
  1    506.637   0.000   0.000   0.000  100.000    31035                        
  2    170.482   0.000   0.000   0.000  100.000    35037                        
  3     89.347  97.297  75.789  85.207  100.000    32915                        
  4     30.437  93.814  95.789  94.792  100.000    29814                     

## Celular Inference 

In [9]:
import spacy
from spacy import displacy

nlp = spacy.load('./models/NUBI-TV-NER-100/model-best')


In [11]:
evaluate = pd.read_csv('../../TVs.csv') #/home/fede/kaggle/desafio_ia_nlp_nubimetrics/TVs.csv
evaluate.head()

,Unnamed: 0,Encrypted_Seller,Encrypted_Id,Title
0,0,b10532ee783f9545d7bddd8f9da20d2a8d1642a486caf5...,63c875becd6c78649539497bdd134c2c762f610b3616ae...,Smart Tv Noblex Di43x5100x Led Full Hd 43 220v
1,1,aa4dd7ba40c80cf15a859c19f2500e328797e2251322d4...,490e4b75cfa66087c95b20fb6546e4dfdc952f7537a8e3...,Smart Tv Tcl L42s6500 Led Full Hd 42 220v
2,2,e48526711c7cd386af46750540bf107acac9b5988b515a...,eab7301d49cd3bbe3bcd3c196da0d61713de45b44523c1...,Tv Samsung Smart Tv Hd 32 Para Repuestos
3,3,9e527f7876123f50719291aed05c8351bab1ae9abacadb...,daf900de69964f2cf241ed5395ac74edd1f2deb6577356...,Tv Led Samsung 46 Smart Pantalla Rota
4,4,cf3021aa458178431fa2f22daa52ee17f005d6728d9b35...,b244c68806fab7d7d523a3927a4eb6381610c40313e8bb...,Tv 29 Sanyo Vizon


In [12]:
marca = []
start = []
end = []

for doc in nlp.pipe(evaluate['Title'][:1000]):    
    #displacy.render(doc, style="ent")
    ent_count = 0
    for ent in doc.ents:
        #print(doc.text, ent.text, ent.start_char, ent.end_char)
        if ent_count == 1:
            pass
        else:        
            marca.append(ent.text)
            start.append(ent.start_char)
            end.append(ent.end_char)
            ent_count = 1
    if ent_count == 0:    
        marca.append('##REV')
        start.append('##REV')
        end.append('##REV')


In [13]:
len(marca)

1000

In [14]:
df_1000 = evaluate.loc[:999, :].copy()
df_1000['marca'] = marca
df_1000['start'] = start
df_1000['end'] = end

In [15]:
df_1000.to_csv("./training_data/tv_ner_textcat_1000_corregir.csv", index=False)

## Celular data 1000 examples

In [16]:
data = pd.read_csv('./training_data/tv_ner_textcat_1000.csv')
data.head() 

,Unnamed: 0,Encrypted_Seller,Encrypted_Id,Title,marca,start,end
0,0,b10532ee783f9545d7bddd8f9da20d2a8d1642a486caf5...,63c875becd6c78649539497bdd134c2c762f610b3616ae...,Smart Tv Noblex Di43x5100x Led Full Hd 43 220v,Noblex,9.0,15.0
1,1,aa4dd7ba40c80cf15a859c19f2500e328797e2251322d4...,490e4b75cfa66087c95b20fb6546e4dfdc952f7537a8e3...,Smart Tv Tcl L42s6500 Led Full Hd 42 220v,Tcl,9.0,12.0
2,2,e48526711c7cd386af46750540bf107acac9b5988b515a...,eab7301d49cd3bbe3bcd3c196da0d61713de45b44523c1...,Tv Samsung Smart Tv Hd 32 Para Repuestos,Samsung,3.0,10.0
3,3,9e527f7876123f50719291aed05c8351bab1ae9abacadb...,daf900de69964f2cf241ed5395ac74edd1f2deb6577356...,Tv Led Samsung 46 Smart Pantalla Rota,Samsung,7.0,14.0
4,4,cf3021aa458178431fa2f22daa52ee17f005d6728d9b35...,b244c68806fab7d7d523a3927a4eb6381610c40313e8bb...,Tv 29 Sanyo Vizon,Sanyo,6.0,11.0


In [17]:
TRAINING_DATA = []

for index, row in data.iterrows(): 
    text = row['Title']
    start = row['start']
    end = row['end']
    entities = []
    #print(start)
    if math.isnan(start):
        pass
        #print('no ent')
    else:        
        entities.append((int(start), int(end), "MARCA"))            
        
    
    #print(dup_start)
    training_example = (text, {"entities": entities})
    TRAINING_DATA.append(training_example)

print(len(TRAINING_DATA))

1000


In [18]:
import random
#TRAINING_DATA
random.seed(0)
l = list(TRAINING_DATA)
random.shuffle(l)
#print(l)
length = len(TRAINING_DATA)
index_80 = length*0.80
train = l[:int(index_80)]
dev = l[int(index_80):]
print(len(train))
print(len(dev))

800
200


In [20]:
import spacy
import srsly
from spacy.gold import docs_to_json, biluo_tags_from_offsets, spans_from_biluo_tags
from tqdm import tqdm


nlp = spacy.load('es_core_news_sm', disable = ['tagger', 'ner'])
#nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp.max_length = 1500000

def spacy_format(data, test_train):    
    docs = []
    for text, annot in tqdm(data):
        if len(text) < 1000000:
            doc = nlp(text)
            tags = biluo_tags_from_offsets(doc, annot['entities'])
            entities = spans_from_biluo_tags(doc, tags)
            doc.ents = entities
            docs.append(doc)

    srsly.write_json(f'./training_data/tv_ner_1000_{test_train}.json', [docs_to_json(docs)]) 

In [21]:
spacy_format(train, 'train')
spacy_format(dev, 'dev')

  0%|          | 0/800 [00:00<?, ?it/s]/tmp/ipykernel_11132/3594294779.py:16: UserWarning: [W030] Some entities could not be aligned in the text "Tv 14 Hitachi/ Broksonic/ Admiral/ Top House/ Vari..." with entities "[(15, 24, 'MARCA')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tags = biluo_tags_from_offsets(doc, annot['entities'])
100%|██████████| 200/200 [00:00<00:00, 780.49it/s]


# Debug

In [22]:
!python3 -m spacy debug-data es '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/TVs/training_data/tv_ner_1000_train.json' '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/TVs/training_data/tv_ner_1000_dev.json' --verbose


=========================== Data format validation ===========================
✔ Corpus is loadable

=============================== Training stats ===============================
Training pipeline: tagger, parser, ner
Starting with blank model 'es'
800 training docs
200 evaluation docs
⚠ 9 training examples also in evaluation data
⚠ Low number of examples to train from a blank model (800)
It's recommended to use at least 2000 examples (minimum 100)

============================== Vocab & Vectors ==============================
ℹ 6066 total words in the data (845 unique)
10 most common words: 'Tv' (587), 'Smart' (319), 'Led' (260), ' ' (254), '32'
(192), 'Pulgadas' (187), 'Televisor' (179), 'Samsung' (154), 'Hd' (147), '4k'
(121)
ℹ No word vectors present in the model

========================== Named Entity Recognition ==========================
ℹ 1 new label, 0 existing labels
0 missing values (tokens with '-' label)
New: 'MARCA' (737)
✔ Good amount of examples for all labels
✔ Examp

# Train 1000 data

In [23]:
!python -m spacy train es ./models/NUBI-TV-NER-1000 '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/TVs/training_data/tv_ner_1000_train.json' '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/TVs/training_data/tv_ner_1000_dev.json' --base-model 'es_core_news_sm' -p ner -R

✔ Created output directory: NUBI-TV-NER-1000
Training pipeline: ['ner']
Starting with base model 'es_core_news_sm'
Replacing component from base model 'ner'
Counting training words (limit=0)
/home/fede/anaconda3/envs/spacy_env/lib/python3.8/site-packages/spacy/language.py:635: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  proc.begin_training(

Itn  NER Loss   NER P   NER R   NER F   Token %  CPU WPS
---  ---------  ------  ------  ------  -------  -------
  1    977.767  92.896  92.896  92.896  100.000    31970                        
  2     85.034  95.652  96.175  95.913  100.000    32535                        
  3     38.335  97.238  96.175  96.703  100.000    33369                        
  4     28.701  98.333  96.721  97.521  100.000    33015                        
  

# Inference 1000

In [25]:
import spacy
from spacy import displacy

nlp = spacy.load('./models/NUBI-TV-NER-1000/model-best')

In [26]:
evaluate = pd.read_csv('../../TVs.csv')
evaluate.head()

,Unnamed: 0,Encrypted_Seller,Encrypted_Id,Title
0,0,b10532ee783f9545d7bddd8f9da20d2a8d1642a486caf5...,63c875becd6c78649539497bdd134c2c762f610b3616ae...,Smart Tv Noblex Di43x5100x Led Full Hd 43 220v
1,1,aa4dd7ba40c80cf15a859c19f2500e328797e2251322d4...,490e4b75cfa66087c95b20fb6546e4dfdc952f7537a8e3...,Smart Tv Tcl L42s6500 Led Full Hd 42 220v
2,2,e48526711c7cd386af46750540bf107acac9b5988b515a...,eab7301d49cd3bbe3bcd3c196da0d61713de45b44523c1...,Tv Samsung Smart Tv Hd 32 Para Repuestos
3,3,9e527f7876123f50719291aed05c8351bab1ae9abacadb...,daf900de69964f2cf241ed5395ac74edd1f2deb6577356...,Tv Led Samsung 46 Smart Pantalla Rota
4,4,cf3021aa458178431fa2f22daa52ee17f005d6728d9b35...,b244c68806fab7d7d523a3927a4eb6381610c40313e8bb...,Tv 29 Sanyo Vizon


In [27]:
marca = []
start = []
end = []

for doc in nlp.pipe(evaluate['Title']):    
    #displacy.render(doc, style="ent")
    ent_count = 0
    for ent in doc.ents:
        #print(doc.text, ent.text, ent.start_char, ent.end_char)
        if ent_count == 1:
            pass
        else:        
            marca.append(ent.text)
            start.append(ent.start_char)
            end.append(ent.end_char)
            ent_count = 1
    if ent_count == 0:    
        marca.append('##REV')
        start.append('##REV')
        end.append('##REV')

In [14]:
df_all = evaluate.loc[:, :].copy()
df_all['marca'] = marca
df_all['start'] = start
df_all['end'] = end

In [15]:
df_all.to_csv("./bd_tv_ner_all.csv", index=False)

## Celular data 1077 examples (Corregidos luego de la inferencia)

In [2]:
data = pd.read_csv('./training_data/tv_ner_textcat_1077.csv')
data.head() 

,Unnamed: 0,Encrypted_Seller,Encrypted_Id,Title,marca,start,end
0,0,b10532ee783f9545d7bddd8f9da20d2a8d1642a486caf5...,63c875becd6c78649539497bdd134c2c762f610b3616ae...,Smart Tv Noblex Di43x5100x Led Full Hd 43 220v,Noblex,9.0,15.0
1,1,aa4dd7ba40c80cf15a859c19f2500e328797e2251322d4...,490e4b75cfa66087c95b20fb6546e4dfdc952f7537a8e3...,Smart Tv Tcl L42s6500 Led Full Hd 42 220v,Tcl,9.0,12.0
2,2,e48526711c7cd386af46750540bf107acac9b5988b515a...,eab7301d49cd3bbe3bcd3c196da0d61713de45b44523c1...,Tv Samsung Smart Tv Hd 32 Para Repuestos,Samsung,3.0,10.0
3,3,9e527f7876123f50719291aed05c8351bab1ae9abacadb...,daf900de69964f2cf241ed5395ac74edd1f2deb6577356...,Tv Led Samsung 46 Smart Pantalla Rota,Samsung,7.0,14.0
4,4,cf3021aa458178431fa2f22daa52ee17f005d6728d9b35...,b244c68806fab7d7d523a3927a4eb6381610c40313e8bb...,Tv 29 Sanyo Vizon,Sanyo,6.0,11.0


In [3]:
TRAINING_DATA = []

for index, row in data.iterrows(): 
    text = row['Title']
    start = row['start']
    end = row['end']
    entities = []
    #print(start)
    if math.isnan(start):
        pass
        #print('no ent')
    else:        
        entities.append((int(start), int(end), "MARCA"))            
        
    
    #print(dup_start)
    training_example = (text, {"entities": entities})
    TRAINING_DATA.append(training_example)

print(len(TRAINING_DATA))

1077


In [4]:
import random
#TRAINING_DATA
random.seed(0)
l = list(TRAINING_DATA)
random.shuffle(l)
#print(l)
length = len(TRAINING_DATA)
index_80 = length*0.80
train = l[:int(index_80)]
dev = l[int(index_80):]
print(len(train))
print(len(dev))

861
216


In [5]:
import spacy
import srsly
from spacy.gold import docs_to_json, biluo_tags_from_offsets, spans_from_biluo_tags
from tqdm import tqdm


nlp = spacy.load('es_core_news_sm', disable = ['tagger', 'ner'])
#nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp.max_length = 1500000

def spacy_format(data, test_train):    
    docs = []
    for text, annot in tqdm(data):
        if len(text) < 1000000:
            doc = nlp(text)
            tags = biluo_tags_from_offsets(doc, annot['entities'])
            entities = spans_from_biluo_tags(doc, tags)
            doc.ents = entities
            docs.append(doc)

    srsly.write_json(f'./training_data/cel_ner_1077_{test_train}.json', [docs_to_json(docs)])

In [6]:
spacy_format(train, 'train')
spacy_format(dev, 'dev')

 62%|██████▏   | 535/861 [00:00<00:00, 779.54it/s]/tmp/ipykernel_14796/3468165033.py:16: UserWarning: [W030] Some entities could not be aligned in the text "Tv 14 Hitachi/ Broksonic/ Admiral/ Top House/ Vari..." with entities "[(15, 24, 'MARCA')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tags = biluo_tags_from_offsets(doc, annot['entities'])
100%|██████████| 216/216 [00:00<00:00, 793.02it/s]


In [7]:
# Debug
!python3 -m spacy debug-data es '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/TVs/training_data/cel_ner_1077_train.json' '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/TVs/training_data/cel_ner_1077_dev.json' --verbose


=========================== Data format validation ===========================
✔ Corpus is loadable

=============================== Training stats ===============================
Training pipeline: tagger, parser, ner
Starting with blank model 'es'
861 training docs
216 evaluation docs
⚠ 9 training examples also in evaluation data
⚠ Low number of examples to train from a blank model (861)
It's recommended to use at least 2000 examples (minimum 100)

============================== Vocab & Vectors ==============================
ℹ 6622 total words in the data (957 unique)
10 most common words: 'Tv' (625), 'Smart' (339), ' ' (275), 'Led' (274),
'Pulgadas' (204), 'Televisor' (200), '32' (186), 'Samsung' (155), 'Hd' (154),
'4k' (123)
ℹ No word vectors present in the model

========================== Named Entity Recognition ==========================
ℹ 1 new label, 0 existing labels
0 missing values (tokens with '-' label)
New: 'MARCA' (779)
✔ Good amount of examples for all labels
✔ Examp

In [8]:
# Train 1077 data
!python -m spacy train es ./models/NUBI-TV-NER-1077 '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/TVs/training_data/cel_ner_1077_train.json' '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/TVs/training_data/cel_ner_1077_dev.json' --base-model 'es_core_news_sm' -p ner -R

✔ Created output directory: models/NUBI-TV-NER-1077
Training pipeline: ['ner']
Starting with base model 'es_core_news_sm'
Replacing component from base model 'ner'
Counting training words (limit=0)
/home/fede/anaconda3/envs/spacy_env/lib/python3.8/site-packages/spacy/language.py:635: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  proc.begin_training(

Itn  NER Loss   NER P   NER R   NER F   Token %  CPU WPS
---  ---------  ------  ------  ------  -------  -------
  1   1011.988  91.753  94.180  92.950  100.000    32877                        
  2     94.645  96.335  97.354  96.842  100.000    33175                        
  3     53.151  95.855  97.884  96.859  100.000    30219                        
  4     22.009  96.335  97.354  96.842  100.000    32944                    

In [9]:
#Inference 1077
import spacy
from spacy import displacy

nlp = spacy.load('./models/NUBI-TV-NER-1077/model-best')
evaluate = pd.read_csv('../../TVs.csv')
evaluate.head()

,Unnamed: 0,Encrypted_Seller,Encrypted_Id,Title
0,0,b10532ee783f9545d7bddd8f9da20d2a8d1642a486caf5...,63c875becd6c78649539497bdd134c2c762f610b3616ae...,Smart Tv Noblex Di43x5100x Led Full Hd 43 220v
1,1,aa4dd7ba40c80cf15a859c19f2500e328797e2251322d4...,490e4b75cfa66087c95b20fb6546e4dfdc952f7537a8e3...,Smart Tv Tcl L42s6500 Led Full Hd 42 220v
2,2,e48526711c7cd386af46750540bf107acac9b5988b515a...,eab7301d49cd3bbe3bcd3c196da0d61713de45b44523c1...,Tv Samsung Smart Tv Hd 32 Para Repuestos
3,3,9e527f7876123f50719291aed05c8351bab1ae9abacadb...,daf900de69964f2cf241ed5395ac74edd1f2deb6577356...,Tv Led Samsung 46 Smart Pantalla Rota
4,4,cf3021aa458178431fa2f22daa52ee17f005d6728d9b35...,b244c68806fab7d7d523a3927a4eb6381610c40313e8bb...,Tv 29 Sanyo Vizon


In [10]:
marca = []
start = []
end = []

for doc in nlp.pipe(evaluate['Title']):    
    #displacy.render(doc, style="ent")
    ent_count = 0
    for ent in doc.ents:
        #print(doc.text, ent.text, ent.start_char, ent.end_char)
        if ent_count == 1:
            pass
        else:        
            marca.append(ent.text)
            start.append(ent.start_char)
            end.append(ent.end_char)
            ent_count = 1
    if ent_count == 0:    
        marca.append('##REV')
        start.append('##REV')
        end.append('##REV')

In [11]:
df_all = evaluate.loc[:, :].copy()
df_all['marca'] = marca
df_all['start'] = start
df_all['end'] = end

In [12]:
df_all.to_csv("./final_bd_tv_ner_all.csv", index=False)